<a href="https://colab.research.google.com/github/KevinYih/BigDataDemo/blob/main/Copy_of_SVD_Netflix_movie_ratings_by_Kevin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SVD - Netflix movie ratings

Adapted from: https://colab.research.google.com/github/ashishdasari148/Recommender-Systems-using-Collaborative-Filtering/blob/master/Netflix.ipynb#scrollTo=qX7Nk4tUGZND

In [1]:
!echo '{"username":"kevinyih","key":"9a18a73669b892b2ac58ebb67dc96545"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d netflix-inc/netflix-prize-data

/bin/bash: line 1: /root/.kaggle/kaggle.json: No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data
License(s): other
 99% 677M/683M [00:06<00:00, 122MB/s]
100% 683M/683M [00:06<00:00, 109MB/s]


In [2]:
import os
import zipfile
with zipfile.ZipFile('netflix-prize-data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
import pandas as pd

import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
dir_name = 'data'

In [4]:
# Converting the txt files into csv files
files = ['combined_data_1.txt','combined_data_2.txt','combined_data_3.txt','combined_data_4.txt']
for j in files:
    file1 = open(os.path.join(dir_name, j),"r+")
    print(j + " opened...")
    file2 = open(os.path.join(dir_name, j + '.csv'),"a")
    for i in file1.readlines():
        temp = re.match("(\d*)[:]", i)
        if(temp):
            movienumber = temp.group(1)
        else:
            file2.write(str(movienumber)+","+i)
    file1.close()
    file2.close()
    print(j + " closed...")

combined_data_1.txt opened...
combined_data_1.txt closed...
combined_data_2.txt opened...
combined_data_2.txt closed...
combined_data_3.txt opened...
combined_data_3.txt closed...
combined_data_4.txt opened...
combined_data_4.txt closed...


In [5]:
movie_titles = pd.read_csv(os.path.join(dir_name, 'movie_titles.csv'), encoding = 'ISO-8859-1', header = None, names=['Movie', 'Year of Release', 'Title'], usecols=[0, 1, 2])
movie_titles

,Movie,Year of Release,Title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [6]:
## appending all the csv files of rating data into a single dataframe
dfs = []
for i in ['combined_data_1.txt.csv','combined_data_2.txt.csv','combined_data_3.txt.csv','combined_data_4.txt.csv']:
    df_ = pd.read_csv(os.path.join(dir_name, i), header=None, names=['Movie', 'User', 'Rating', 'Date'], usecols=[0, 1, 2, 3])
    df_ = df_[['Movie','User','Rating']]
    dfs.append(df_)
df_all = pd.concat(dfs, ignore_index=True)

print(df_all.head(5))

   Movie     User  Rating
0      1  1488844       3
1      1   822109       5
2      1   885013       4
3      1    30878       4
4      1   823519       3


In [7]:
print("number of ratings:", len(df_all))

number of ratings: 100480507


In [26]:
# search for your favorite movie

indices = movie_titles.Title.str.contains('dinosaur', flags=re.IGNORECASE)
movie_titles[indices]
dinosaur_movie_ids = set(movie_titles[indices].Movie)
dinosaur_movie_ids

# 14691    The Matrix
# 3925     The Matrix: Reloaded
# 16767    The Matrix: Revisited
# 468      The Matrix: Revolutions
# 397      A Night in Casablanca
# 10661    Tokyo Story
# 963      A Streetcar Named Desire
# 6099     Apocalypse Now
# 4253    Kind Hearts and Coronets
# 2412     Harry Potter and the Prisoner of Azkaban: Bonu...
# 11443    Harry Potter and the Chamber of Secrets



{1, 651, 694, 1173, 1883, 4100, 5302, 8842, 13586, 13891, 15648}

In [28]:
# choosing random movies
random_movie_ids = set(df_all.sample(frac=.0000001).Movie.unique())
print(random_movie_ids)

# or choosing your favorite
favorites_movie_ids = set({1, 651, 694, 1173, 1883, 4100, 5302, 8842, 13586, 13891, 15648})
print(favorites_movie_ids)

{5345, 2913, 12355, 9188, 11397, 12593, 6642, 10162, 11165, 10078}
{15648, 1, 13891, 4100, 8842, 651, 13586, 1173, 694, 5302, 1883}


In [29]:
df_all[df_all.Movie == 17339 ]

,Movie,User,Rating
98172243,17339,1734805,3
98172244,17339,364518,4
98172245,17339,1392773,3
98172246,17339,716091,4
98172247,17339,2059652,4
...,...,...,...
98243197,17339,702455,3
98243198,17339,1536433,3
98243199,17339,1704768,3
98243200,17339,2486011,4


In [35]:
# saving the merged file to drive for further use.

movie_ids = random_movie_ids.union(favorites_movie_ids)

df_all.to_csv(path_or_buf=os.path.join(dir_name, 'data.txt.csv'),index=False)

In [16]:
# loading the saved csv into dataframe
df_all = pd.read_csv(os.path.join(dir_name, 'data.txt.csv'))

In [36]:
print("number of ratings:", len(df_all))

number of ratings: 100480507


In [37]:
df_movie_ids = df_all[df_all.Movie.isin(movie_ids)]
print("Unique movie ids in the dataframe:", (df_movie_ids.Movie.unique()))
print("Number of unique user ids:", len(df_movie_ids.User.unique()))

Unique movie ids in the dataframe: [    1   651   694  1173  1883  2913  4100  5302  5345  6642  8842  9188
 10078 10162 11165 11397 12355 12593 13586 13891 15648]
Number of unique user ids: 241936


In [38]:
df_ = df_movie_ids.sample(frac=0.001)
len(df_)

518

In [39]:
# create a dataframe that only contains rating for a subset of users and
# a subset of movies
user_ids = set(df_.User.unique())
df_small = df_all[(df_all.User.isin(user_ids) & df_all.Movie.isin(movie_ids))]
print("Unique movie ids in the dataframe:", (df_small.Movie.unique()))
print("Number of unique user ids:", len(df_small.User.unique()))
print("Number of ratings:", len(df_small))

Unique movie ids in the dataframe: [    1   651   694  1173  1883  2913  4100  5302  5345  6642  8842  9188
 10078 10162 11165 11397 12355 12593 13586 13891 15648]
Number of unique user ids: 517
Number of ratings: 1605


In [40]:
itom = list(sorted(movie_ids))
mtoi = {m: i for i, m in enumerate(itom)}
itou = list(sorted(user_ids))
utoi = {u: i for i, u in enumerate(itou)}

In [41]:
user_movie_rating = np.zeros((len(itou), len(itom)))

for index, row in df_small.iterrows():
  movie_id = row.Movie
  movie_index = mtoi[movie_id]

  user_id = row.User
  user_index = utoi[user_id]

  user_movie_rating[user_index, movie_index] = row.Rating

In [42]:
user_movie_rating.shape

(517, 21)

In [43]:
# more efficient way of building a user-movie rating matrix
um = df_small.pivot_table(index='User', columns='Movie', values='Rating').values
um.shape

(517, 21)

In [44]:
user_movie_rating[:10,]

array([[0., 0., 0., 0., 0., 3., 0., 0., 3., 0., 0., 3., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 4., 0., 0., 3., 0., 0., 4., 3., 0., 0., 0.,
        3., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 3., 4., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 4., 4., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 0., 0., 3., 4., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 3., 0., 0., 0., 4., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 4., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.,

In [45]:
# compute the SVD of the matrix
A = user_movie_rating
U, S, Vt = np.linalg.svd(A, full_matrices=False)

In [46]:
print(U.shape, S.shape, Vt.shape)

(517, 21) (21,) (21, 21)


In [47]:
A_hat = U @ np.diag(S) @ Vt

In [48]:
np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat[:10,])

[[0.00 -0.00 0.00 -0.00 -0.00 3.00 -0.00 -0.00 3.00 -0.00 -0.00 3.00
  -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00]
 [0.00 -0.00 0.00 -0.00 -0.00 4.00 -0.00 0.00 3.00 -0.00 -0.00 4.00 3.00
  -0.00 -0.00 -0.00 3.00 -0.00 -0.00 -0.00 -0.00]
 [-0.00 -0.00 0.00 -0.00 0.00 0.00 -0.00 -0.00 4.00 -0.00 -0.00 0.00
  -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00]
 [0.00 -0.00 -0.00 0.00 -0.00 3.00 0.00 -0.00 0.00 0.00 -0.00 -0.00 3.00
  4.00 -0.00 -0.00 -0.00 -0.00 0.00 -0.00 -0.00]
 [0.00 -0.00 0.00 0.00 -0.00 3.00 0.00 -0.00 0.00 -0.00 0.00 4.00 4.00
  -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00]
 [-0.00 -0.00 0.00 -0.00 -0.00 5.00 -0.00 0.00 3.00 4.00 -0.00 -0.00
  -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 -0.00 -0.00 3.00 0.00 0.00 0.00 4.00 -0.00 2.00 -0.00
  -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 0.00 -0.00 -0.00 0.00 -0.00 -0.00 0.00 -0.00 0.00 4.00 4.00
  -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 0.00 -0.00]
 [0.00 0.00

In [49]:
sum([a*a for a in S[:3]])/sum([a*a for a in S])

0.6550040191135936

In [50]:
r = 8
A_hat_r = U[:,:r] @ np.diag(S[:r]) @ Vt[:r,:]
np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat_r[:10,])

[[0.03 0.00 -0.03 -0.09 0.02 3.00 0.08 -0.00 2.99 0.01 -0.01 3.00 -0.01
  0.03 0.00 -0.00 -0.01 -0.04 -0.07 0.00 0.01]
 [0.11 0.03 0.11 0.23 0.01 4.06 1.20 0.04 2.80 0.19 0.03 4.12 2.77 -0.10
  0.12 0.13 2.54 -0.04 0.17 0.01 0.01]
 [0.07 0.02 0.00 0.01 0.03 0.02 0.54 0.03 3.91 0.08 0.02 0.05 -0.10 -0.01
  0.05 -0.03 -0.19 -0.05 0.02 0.01 0.00]
 [0.11 -0.02 0.21 0.22 -0.01 3.04 0.48 -0.02 -0.07 0.08 -0.00 0.06 2.84
  2.92 0.01 0.24 -0.21 1.74 0.08 -0.01 -0.01]
 [0.03 -0.02 0.04 -0.07 -0.02 3.01 0.17 -0.03 -0.03 0.02 -0.06 4.01 3.97
  -0.06 0.01 0.09 -0.05 0.07 -0.02 -0.00 0.01]
 [-0.04 -0.01 -0.09 -0.16 0.01 4.99 -0.22 -0.02 3.04 3.96 -0.03 -0.03
  0.05 0.06 -0.03 -0.07 0.11 -0.03 -0.04 -0.00 -0.01]
 [-0.10 -0.02 -0.12 -0.22 -0.01 2.97 -0.69 -0.03 0.12 3.89 -0.06 1.92
  0.13 0.07 -0.08 -0.03 0.28 0.02 -0.09 -0.00 -0.01]
 [0.04 -0.01 0.04 -0.07 -0.01 0.01 0.28 -0.02 -0.04 0.04 -0.05 4.02 3.95
  -0.08 0.02 0.09 -0.09 0.09 -0.01 0.00 0.00]
 [0.06 -0.01 0.09 0.05 -0.01 0.03 0.57 -0.01 -0.10

In [54]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7beff610998a9613620316e9e60e333c436de658ed046f72446fb492610721ab
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [57]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import numpy as np
import os

spark = SparkSession.builder.appName("MovieRatings").getOrCreate()

In [56]:
df_all = spark.read.csv(os.path.join(dir_name, 'data.txt.csv'), header=True, inferSchema=True)

movie_ids = random_movie_ids.union(favorites_movie_ids)

df_movie_ids = df_all.filter(col('Movie').isin(list(movie_ids)))

df_sample = df_movie_ids.sample(fraction=0.001)

user_ids = set(df_sample.select('User').distinct().rdd.flatMap(lambda x: x).collect())

df_small = df_all.filter((col('User').isin(list(user_ids))) & (col('Movie').isin(list(movie_ids))))

itom = list(sorted(movie_ids))
mtoi = {m: i for i, m in enumerate(itom)}
itou = list(sorted(user_ids))
utoi = {u: i for i, u in enumerate(itou)}

user_movie_rating = np.zeros((len(itou), len(itom)))

for row in df_small.collect():
    movie_id = row['Movie']
    movie_index = mtoi[movie_id]
    user_id = row['User']
    user_index = utoi[user_id]
    user_movie_rating[user_index, movie_index] = row['Rating']

print(user_movie_rating.shape)

A = user_movie_rating
U, S, Vt = np.linalg.svd(A, full_matrices=False)

A_hat = U @ np.diag(S) @ Vt

np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat[:10,])

sum_S_square = sum([a * a for a in S])
sum_S_square_3 = sum([a * a for a in S[:3]])
print(sum_S_square_3 / sum_S_square)

r = 8
A_hat_r = U[:, :r] @ np.diag(S[:r]) @ Vt[:r, :]
np.set_printoptions(suppress=True, formatter={'float': '{:0.2f}'.format})
print(A_hat_r[:10,])


(528, 21)
[[0.00 -0.00 -0.00 0.00 0.00 3.00 0.00 0.00 3.00 0.00 0.00 -0.00 -0.00
  3.00 4.00 0.00 0.00 0.00 0.00 0.00 -0.00]
 [0.00 0.00 -0.00 0.00 -0.00 4.00 3.00 -0.00 5.00 4.00 0.00 4.00 5.00
  0.00 2.00 0.00 3.00 3.00 0.00 -0.00 -0.00]
 [0.00 -0.00 -0.00 0.00 0.00 5.00 -0.00 -0.00 4.00 0.00 0.00 -0.00 4.00
  0.00 5.00 -0.00 4.00 0.00 -0.00 0.00 0.00]
 [0.00 0.00 -0.00 -0.00 -0.00 -0.00 -0.00 0.00 3.00 0.00 -0.00 4.00 3.00
  0.00 0.00 0.00 3.00 0.00 -0.00 -0.00 -0.00]
 [-0.00 0.00 0.00 -0.00 0.00 -0.00 -0.00 0.00 -0.00 0.00 -0.00 2.00 -0.00
  0.00 -0.00 0.00 -0.00 0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 -0.00 -0.00 4.00 -0.00 -0.00 -0.00 -0.00 0.00 -0.00
  -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00 -0.00]
 [0.00 0.00 3.00 -0.00 -0.00 -0.00 3.00 0.00 0.00 0.00 0.00 0.00 -0.00
  0.00 5.00 2.00 -0.00 -0.00 3.00 -0.00 -0.00]
 [-0.00 -0.00 -0.00 0.00 -0.00 3.00 -0.00 -0.00 2.00 3.00 0.00 0.00 0.00
  5.00 5.00 3.00 3.00 4.00 0.00 -0.00 -0.00]
 [0.00 0.00 -0.00 -0.00 0.00 -0.00 0.00